<a href="https://colab.research.google.com/github/Hee0305/DL/blob/main/ll.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
# from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.mobilenet_v2 import  MobileNetV2, preprocess_input
import tensorflow.keras as keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
import tensorflow as tf
from keras.utils import np_utils
from keras.models import load_model
from keras.datasets import cifar10
from keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
# import cv2
print(tf.__version__)

2.7.0


In [53]:
tf.keras.backend.clear_session()  # For memory

# GPU check
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print("#### ", len(gpus), "Physical GPUs,",
                 len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
else:
    print('#### No CUDA supported GPU in this computer.')

####  1 Physical GPUs, 1 Logical GPUs


**Data load**

In [54]:
from keras.datasets import cifar10
# load dataset
(trainX, trainy), (testX, testy) = cifar10.load_data()
# summarize loaded dataset
print('Train: X=%s, y=%s' % (trainX.shape, trainy.shape))
print('Test: X=%s, y=%s' % (testX.shape, testy.shape))

#training의 10% split -> numpy 코드 이용(사이킷런)

Train: X=(50000, 32, 32, 3), y=(50000, 1)
Test: X=(10000, 32, 32, 3), y=(10000, 1)


In [55]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = \
            train_test_split(trainX,trainy,
                             random_state=42,
                             test_size=0.1,
                             stratify=trainy)

Xtrain = X_test
print(Xtrain.shape) # Xtrain
print(testX.shape) # testX
ytrain = y_test
print(ytrain.shape) # ytrain
print(testy.shape) # testy

(5000, 32, 32, 3)
(10000, 32, 32, 3)
(5000, 1)
(10000, 1)


In [56]:
TL_base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [57]:
# len(TL_base.layers)

In [58]:
# TL_base.summary()

In [59]:
TL_base.layers[-1].get_config() # (7, 7, 1280)

{'dtype': 'float32',
 'max_value': array(6., dtype=float32),
 'name': 'out_relu',
 'negative_slope': array(0., dtype=float32),
 'threshold': array(0., dtype=float32),
 'trainable': True}

 **정규화**

In [60]:
Xtrain = Xtrain.astype('float32')
testX = testX.astype('float32')

Xtrain = preprocess_input(Xtrain)
testX = preprocess_input(testX)

# No one-hot encoding
# y_train = np_utils.to_categorical(y_train, 10)
# y_test = np_utils.to_categorical(y_test, 10)

print(Xtrain.shape)
print(testX.shape)


(5000, 32, 32, 3)
(10000, 32, 32, 3)


In [61]:
Xtrain.shape,len(Xtrain),Xtrain[4].shape

((5000, 32, 32, 3), 5000, (32, 32, 3))

In [62]:
# model.summary()

 **Sequential model**

In [63]:
# Set TL_base trainable or not.
TL_base.trainable = False  # Use the representative features pretrained by resnet.

In [64]:
model = models.Sequential()
model.add(keras.Input(shape=(32,32,3)))
model.add(layers.UpSampling2D((7,7)))  # 32*7 = 224
model.add(TL_base)
model.add(layers.GlobalAveragePooling2D()) # (7,7,2048) => (2048,)
model.add(layers.Flatten())
model.add(layers.BatchNormalization())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.BatchNormalization())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.BatchNormalization())
model.add(layers.Dense(10, activation='softmax'))

# model.compile(optimizer=optimizers.RMSprop(lr=2e-5), loss='binary_crossentropy', metrics=['acc'])
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 
model.compile(optimizer='SGD', 
                loss='sparse_categorical_crossentropy',  # sparse_categorical_crossentropy
                metrics = ['accuracy'])

In [65]:
from tensorflow.keras import layers, models, callbacks

In [66]:
mc_callback = callbacks.ModelCheckpoint(filepath="./model/cifar10_mobileNetV2S_SGD_best_weights.{epoch:03d}-{val_accuracy:.4f}.hdf5", 
                              monitor='val_accuracy', verbose=0, save_best_only=True)
es_callback = callbacks.EarlyStopping(monitor='val_accuracy', 
                            mode='max', verbose=1, patience=3)

**Data augmentation**

In [67]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=10,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
    #zoom_range=0.3
    )
datagen.fit(Xtrain)



In [ ]:
epochs = 100
BATCH_SIZE = 32
hist = model.fit(datagen.flow(Xtrain, ytrain, batch_size=BATCH_SIZE), 
                 steps_per_epoch=X_train.shape[0]//BATCH_SIZE, 
                 epochs=epochs, verbose=1, 
                 callbacks=[mc_callback, es_callback], 
                 validation_data=(testX,testy))

Epoch 1/100
 157/1406 [==>...........................] - ETA: 2:10 - loss: 1.8471 - accuracy: 0.0916WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 140600 batches). You may need to use the repeat() function when building your dataset.


In [70]:
model.evaluate(testX, testy, batch_size = BATCH_SIZE)

145/200 [====================>.........] - ETA: 8s - loss: 1.6990 - accuracy: 0.0484

KeyboardInterrupt: ignored

In [ ]:
# history = model.fit(Xtrain, ytrain, 
#                     epochs=100, batch_size=64, 
#                     validation_data=(testX, testy),
#                     callbacks=[mc_callback,es_callback])